**LSDB** main branch!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20
plt.rc('font', family='serif')

import lsdb
#from lsdb import lsdb_client
#client = lsdb_client(dask_on_ray=False, num_workers=12)

In [2]:
#Load ZTF, Gaia, and ZTF sources hipscats
gaia = lsdb.read_hipscat("/data3/epyc/data3/hipscat/test_catalogs/gaia_symbolic")

In [4]:
# load ZTF object table
ztf = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14")

In [9]:
ztf_sources = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_source")

In [24]:
ztf_sources

,index,ps1_objid,ra,dec,ps1_gMeanPSFMag,ps1_rMeanPSFMag,ps1_iMeanPSFMag,nobs_g,nobs_r,nobs_i,mean_mag_g,mean_mag_r,mean_mag_i,catflags,fieldID,mag,magerr,mjd,rcID,band,Norder,Dir,Npix
npartitions=311037,,,,,,,,,,,,,,,,,,,,,,,
0,int64,int64,float64,float64,float64,float64,float64,int32,int32,int32,float64,float64,float64,int16,int16,float32,float32,float64,int16,string,int32,int32,int32
281474976710656,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13834987686537986048,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18446744073709551615,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
%%time
ra, dec = 132, 5
rad = 5 # in degrees
_sample = gaia.cone_search(
    ra=ra,
    dec=dec,
    radius=rad,
).crossmatch(
    ztf
).query(
    "nobs_g_ztf_dr14 > 150 and nobs_r_ztf_dr14 > 150 and \
    parallax_gaia > 0 and parallax_over_error_gaia > 10 and \
    classprob_dsc_combmod_star_gaia > 0.9"
).compute()

/astro/users/atzanida/lsdbgithub/lsdb/src/lsdb/core/cone_search.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_frame["_CONE_SEP"] = df_separations
/astro/users/atzanida/lsdbgithub/lsdb/src/lsdb/core/cone_search.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_frame["_CONE_SEP"] = df_separations
/astro/users/atzanida/lsdbgithub/lsdb/src/lsdb/core/cone_search.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  

CPU times: user 34.3 s, sys: 42.3 s, total: 1min 16s
Wall time: 18.3 s


In [11]:
# Convert crossmatch _sample to lsdb Catalog object
_sample_hips = lsdb.from_dataframe(
    _sample, 
    lowest_order=5, 
    highest_order=8, 
    ra_column="ra_gaia", 
    dec_column="dec_gaia", 
    threshold=1_000_000
)

In [12]:
# join tables to final ZTF source list (i.e photometry)
_sample_sources = _sample_hips.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid"
).compute()

# don't compute table...
ss = _sample_hips.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid"
)

In [25]:
ss

,solution_id_gaia_,designation_gaia_,source_id_gaia_,random_index_gaia_,ref_epoch_gaia_,ra_gaia_,ra_error_gaia_,dec_gaia_,dec_error_gaia_,parallax_gaia_,parallax_error_gaia_,parallax_over_error_gaia_,pm_gaia_,pmra_gaia_,pmra_error_gaia_,pmdec_gaia_,pmdec_error_gaia_,ra_dec_corr_gaia_,ra_parallax_corr_gaia_,ra_pmra_corr_gaia_,ra_pmdec_corr_gaia_,dec_parallax_corr_gaia_,dec_pmra_corr_gaia_,dec_pmdec_corr_gaia_,parallax_pmra_corr_gaia_,parallax_pmdec_corr_gaia_,pmra_pmdec_corr_gaia_,astrometric_n_obs_al_gaia_,astrometric_n_obs_ac_gaia_,astrometric_n_good_obs_al_gaia_,astrometric_n_bad_obs_al_gaia_,astrometric_gof_al_gaia_,astrometric_chi2_al_gaia_,astrometric_excess_noise_gaia_,astrometric_excess_noise_sig_gaia_,astrometric_params_solved_gaia_,astrometric_primary_flag_gaia_,nu_eff_used_in_astrometry_gaia_,pseudocolour_gaia_,pseudocolour_error_gaia_,ra_pseudocolour_corr_gaia_,dec_pseudocolour_corr_gaia_,parallax_pseudocolour_corr_gaia_,pmra_pseudocolour_corr_gaia_,pmdec_pseudocolour_corr_gaia_,astrometric_matched_transits_gaia_,visibility_periods_used_gaia_,astrometric_sigma5d_max_gaia_,matched_transits_gaia_,new_matched_transits_gaia_,matched_transits_removed_gaia_,ipd_gof_harmonic_amplitude_gaia_,ipd_gof_harmonic_phase_gaia_,ipd_frac_multi_peak_gaia_,ipd_frac_odd_win_gaia_,ruwe_gaia_,scan_direction_strength_k1_gaia_,scan_direction_strength_k2_gaia_,scan_direction_strength_k3_gaia_,scan_direction_strength_k4_gaia_,scan_direction_mean_k1_gaia_,scan_direction_mean_k2_gaia_,scan_direction_mean_k3_gaia_,scan_direction_mean_k4_gaia_,duplicated_source_gaia_,phot_g_n_obs_gaia_,phot_g_mean_flux_gaia_,phot_g_mean_flux_error_gaia_,phot_g_mean_flux_over_error_gaia_,phot_g_mean_mag_gaia_,phot_bp_n_obs_gaia_,phot_bp_mean_flux_gaia_,phot_bp_mean_flux_error_gaia_,phot_bp_mean_flux_over_error_gaia_,phot_bp_mean_mag_gaia_,phot_rp_n_obs_gaia_,phot_rp_mean_flux_gaia_,phot_rp_mean_flux_error_gaia_,phot_rp_mean_flux_over_error_gaia_,phot_rp_mean_mag_gaia_,phot_bp_rp_excess_factor_gaia_,phot_bp_n_contaminated_transits_gaia_,phot_bp_n_blended_transits_gaia_,phot_rp_n_contaminated_transits_gaia_,phot_rp_n_blended_transits_gaia_,phot_proc_mode_gaia_,bp_rp_gaia_,bp_g_gaia_,g_rp_gaia_,radial_velocity_gaia_,radial_velocity_error_gaia_,rv_method_used_gaia_,rv_nb_transits_gaia_,rv_nb_deblended_transits_gaia_,rv_visibility_periods_used_gaia_,rv_expected_sig_to_noise_gaia_,rv_renormalised_gof_gaia_,rv_chisq_pvalue_gaia_,rv_time_duration_gaia_,rv_amplitude_robust_gaia_,rv_template_teff_gaia_,rv_template_logg_gaia_,rv_template_fe_h_gaia_,rv_atm_param_origin_gaia_,vbroad_gaia_,vbroad_error_gaia_,vbroad_nb_transits_gaia_,grvs_mag_gaia_,grvs_mag_error_gaia_,grvs_mag_nb_transits_gaia_,rvs_spec_sig_to_noise_gaia_,phot_variable_flag_gaia_,l_gaia_,b_gaia_,ecl_lon_gaia_,ecl_lat_gaia_,in_qso_candidates_gaia_,in_galaxy_candidates_gaia_,non_single_star_gaia_,has_xp_continuous_gaia_,has_xp_sampled_gaia_,has_rvs_gaia_,has_epoch_photometry_gaia_,has_epoch_rv_gaia_,has_mcmc_gspphot_gaia_,has_mcmc_msc_gaia_,in_andromeda_survey_gaia_,classprob_dsc_combmod_quasar_gaia_,classprob_dsc_combmod_galaxy_gaia_,classprob_dsc_combmod_star_gaia_,teff_gspphot_gaia_,teff_gspphot_lower_gaia_,teff_gspphot_upper_gaia_,logg_gspphot_gaia_,logg_gspphot_lower_gaia_,logg_gspphot_upper_gaia_,mh_gspphot_gaia_,mh_gspphot_lower_gaia_,mh_gspphot_upper_gaia_,distance_gspphot_gaia_,distance_gspphot_lower_gaia_,distance_gspphot_upper_gaia_,azero_gspphot_gaia_,azero_gspphot_lower_gaia_,azero_gspphot_upper_gaia_,ag_gspphot_gaia_,ag_gspphot_lower_gaia_,ag_gspphot_upper_gaia_,ebpminrp_gspphot_gaia_,ebpminrp_gspphot_lower_gaia_,ebpminrp_gspphot_upper_gaia_,libname_gspphot_gaia_,Norder_gaia_,Npix_gaia_,Dir_gaia_,ps1_objid_ztf_dr14_,ra_ztf_dr14_,dec_ztf_dr14_,ps1_gMeanPSFMag_ztf_dr14_,ps1_rMeanPSFMag_ztf_dr14_,ps1_iMeanPSFMag_ztf_dr14_,nobs_g_ztf_dr14_,nobs_r_ztf_dr14_,nobs_i_ztf_dr14_,mean_mag_g_ztf_dr14_,mean_mag_r_ztf_dr14_,mean_mag_i_ztf_dr14_,Norder_ztf_dr14_,Dir_ztf_dr14_,Npix_ztf_dr14_,_DIST_,Norder_,Dir_,Np

In [13]:
import dask.dataframe as dd
from tape import Ensemble, ColumnMapper

In [14]:
# Initialize an Ensemble
ens = Ensemble()
ens.client_info()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 96,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34092,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 96
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:39982,Total threads: 8
Dashboard: http://127.0.0.1:40974/status,Memory: 83.97 GiB
Nanny: tcp://127.0.0.1:36301,


In [26]:
# ColumnMapper Establishes which table columns map to timeseries quantities
colmap = ColumnMapper(
        id_col='_hipscat_index',
        time_col='mjd_ztf_source',
        flux_col='mag',
        err_col='magerr',
        band_col='band',
      )

In [27]:
# We can read from parquet
ens.from_dask_dataframe(
    source_frame=ss._ddf,
    object_frame=_sample_hips._ddf,
    column_mapper=colmap,
    sync_tables=False, # Avoid doing an initial sync
    sorted=True, # If the input data is already sorted by the chosen index
    sort=False,
)

/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/dataframe/core.py:5400: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/dataframe/core.py:5400: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


In [28]:
import astropy.stats as astro_stats
def deviation(mag, mag_err):
    """Calculate the running deviation of a light curve for outburst or dip detection.
    
    d >> 0 will be dimming
    d << 0 (or negative) will be brightenning
    
    
    Parameters:
    -----------
    mag (array-like): Magnitude values of the light curve.
    mag_err (array-like): Magnitude errors of the light curve.

    Returns:
    --------
    dev (array-like): Deviation values of the light curve.
    """
    # Calculate biweight estimators
    R, S = astro_stats.biweight_location(mag), astro_stats.biweight_scale(mag)

    return (mag - R) / np.sqrt(mag_err**2 + S**2)


def frac_above_dev(dev):
    """Calculate the fraction of detections above 2 sigma from the deviation"""
    M = np.median(dev)
    # fraction above 2-sigma from the M
    two_sig = M + 2*np.std(dev)
    
    if len(dev)>0:
        return len(dev[dev>two_sig])/len(dev)
    else:
        return 0


def assymetry_yso_M(mag):
    """Calculate the magnitude assymetry score defined by Hillenbrand et al. 2022 (https://iopscience.iop.org/article/10.3847/1538-3881/ac62d8/pdf).

    Described in the paper: 
    Objects that have M values <0 are predominately brightening,
    objects with M values >0 are predominantly dimming, and
    objects with M values near 0 have symmetric light curves.

    Parameters:
    -----------
    mag (array-like): Magnitude values of the light curve.
    
    Returns:
    --------
    assymetry (float): Assymetry score.
    """
    
    if len(mag)>0:
        mag_decile = np.percentile(mag, 10)

        return (mag_decile - np.nanmedian(mag))/np.nanstd(mag)
    else:
        return 0

In [29]:
# output columns
output_cols = ["median", "frac_dev", "M_stat"]

# Define DataFrame with loc and scale as meta
my_meta = pd.DataFrame(columns=output_cols, dtype=float)

def calc_biweight_lc(time, mag, mag_err, flag, band):
    """custom function"""
    rmv = (flag == 0) & (band=='r')
    
    time, mag, mag_err, flag, band = time[rmv], mag[rmv], mag_err[rmv], flag[rmv], band[rmv]
    
    # sort times
    xs = np.argsort(time)
    
    time, mag, mag_err, flag = time[xs], mag[xs], mag_err[xs], flag[xs]
    
    # compute features
    _f = frac_above_dev(deviation(mag, mag_err))
    _M = assymetry_yso_M(mag)
    
    return pd.Series([np.median(mag), _f, _M],
                     index=output_cols)


In [32]:
%%time
# apply calc_biweight function
calc_ = ens.batch(
    calc_biweight_lc,
    'mjd_ztf_source', 'mag_ztf_source', 'magerr_ztf_source', 'catflags_ztf_source','band_ztf_source',
    meta=my_meta,
    use_map=True, 
    compute=False
)

CPU times: user 19.9 ms, sys: 30.2 ms, total: 50.1 ms
Wall time: 47.3 ms


In [33]:
calc_.compute()

/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 93.66 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of

,median,frac_dev,M_stat
_hipscat_index,,,
1153108924551921664,18.469278,0.011952,-1.069675
1153108975441412096,17.395103,0.007968,-1.278913
1153109401842745344,18.357708,0.024096,-1.276625
1153109493127577600,12.441140,0.035533,-1.033353
1153109506784231424,15.588398,0.008163,-1.253393
...,...,...,...
6187368344409079808,16.893934,0.032110,-0.855995
6187368651037868032,17.451550,0.007533,-0.874071
6187368657866194944,17.306484,0.010121,-0.944448


In [34]:
ens.assign(table='object', med=calc_['median'], frac_dev=calc_["frac_dev"], Mstat=calc_["M_stat"])

In [35]:
ens._object.head(1)

,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,...,Norder_ztf_dr14,Dir_ztf_dr14,Npix_ztf_dr14,_DIST,Norder,Dir,Npix,med,frac_dev,Mstat
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
1153108924551921664,1636148068921376768,Gaia DR3 576554455605646848,576554455605646848,25600780,2016.0,134.371506,0.14738,0.60513,0.078145,1.881222,...,3,0,64,0.000016,5,0,1024,18.469278,0.011952,-1.069675
